In [2]:
import pandas as pd
# Ler o arquivo Parquet
df = pd.read_parquet('Dados/df_ativas_ultimo_trimestre_2023.parquet')
df.head()

,CNPJ,UF,MUNICIPIO,CEP,DataSituacaoCadastral,DataCadastro,Porte,CNAE_Principal,CNAE_Secundario,MUN_COD_IBGE,DESC_MUNICIPIO,COD_RGI,NOME_RGI,Setor,Região,Descricao_CNAE_Principal,CNAE_Divisão,Divisão_Descrição
0,42804324000394,SP,2941,15315000,2023-11-03,2023-11-03,DEMAIS,0151201,0115600,3549250,São João de Iracema,350029,Fernandópolis,Agropecuaria,Sudeste,Criação de bovinos para corte,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
1,42806249000481,MG,4371,32040027,2023-11-14,2023-11-14,DEMAIS,4635499,"4635402,4723700,4789001",3118601,Contagem,310001,Belo Horizonte,Comercio,Sudeste,Comércio atacadista de bebidas não especificad...,46,"COMÉRCIO POR ATACADO, EXCETO VEÍCULOS AUTOMOTO..."
2,42879032000230,BA,3873,44445450,2023-11-14,2023-11-14,ME,3101200,"4752100,4753900,4754701,4754702",2928703,Santo Antônio de Jesus,290003,Santo Antônio de Jesus,Industria,Nordeste,Fabricação de móveis com predominância de madeira,31,FABRICAÇÃO DE MÓVEIS
3,43952585000233,SP,6839,15630000,2023-11-10,2023-11-10,DEMAIS,0139306,None,3536901,Pedranópolis,350029,Fernandópolis,Agropecuaria,Sudeste,Cultivo de seringueira,01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"
4,43975859000200,SP,6645,12614899,2023-10-05,2023-10-05,DEMAIS,0121101,"0151201,0151202",3527207,Lorena,350052,Guaratinguetá,Agropecuaria,Sudeste,"Horticultura, exceto morango",01,"AGRICULTURA, PECUÁRIA E SERVIÇOS RELACIONADOS"


In [3]:
# Verificando o tipo de cada coluna
print(df.dtypes)

CNPJ                                object
UF                                  object
MUNICIPIO                           object
CEP                                 object
DataSituacaoCadastral       datetime64[us]
DataCadastro                datetime64[us]
Porte                               object
CNAE_Principal                      object
CNAE_Secundario                     object
MUN_COD_IBGE                        object
DESC_MUNICIPIO                      object
COD_RGI                             object
NOME_RGI                            object
Setor                               object
Região                              object
Descricao_CNAE_Principal            object
CNAE_Divisão                        object
Divisão_Descrição                   object
dtype: object


In [4]:
df.Setor.value_counts()

Setor
Servicos            136554
Comercio             49132
Industria            10188
Construcao Civil      8623
Agropecuaria          7027
Name: count, dtype: int64

In [8]:
df.describe()

,DataSituacaoCadastral,DataCadastro
count,211524,211524
mean,2023-11-13 17:51:30.645033,2023-11-13 17:50:51.840926
min,2023-10-02 00:00:00,2023-10-02 00:00:00
25%,2023-10-23 00:00:00,2023-10-23 00:00:00
50%,2023-11-14 00:00:00,2023-11-14 00:00:00
75%,2023-12-06 00:00:00,2023-12-06 00:00:00
max,2023-12-31 00:00:00,2023-12-31 00:00:00


In [10]:
# Filtrar os dados para os portes ME e EPP
df_filtered = df[df['Porte'].isin(['ME', 'EPP'])]

# Listar os setores disponíveis
setores = df_filtered['Setor'].unique()
print("Setores disponíveis:", setores)

Setores disponíveis: ['Industria' 'Servicos' 'Agropecuaria' 'Comercio' 'Construcao Civil']


In [12]:
df_filtered.Setor.value_counts()

Setor
Servicos            109508
Comercio             47740
Industria             8195
Construcao Civil      7781
Agropecuaria          1883
Name: count, dtype: int64

In [29]:
import pandas as pd
import igraph as ig
import time
from itertools import combinations
from tqdm import tqdm
import matplotlib.pyplot as plt

# Função para calcular o peso das arestas
def calcular_peso(row1, row2):
    peso = 0
    if row1['CNAE_Principal'] == row2['CNAE_Principal']:
        peso += 1
    if row1['MUNICIPIO'] == row2['MUNICIPIO']:
        peso += 1
    if row1['Porte'] == row2['Porte']:
        peso += 1
    if row1['COD_RGI'] == row2['COD_RGI']:
        peso += 0.5
    if row1['CNAE_Divisão'] == row2['CNAE_Divisão']:
        peso += 0.5
    return peso

# Função principal para análise da rede
def analisar_rede_por_setor_e_calcular_metricas(df, setor, peso_minimo=None, grau_minimo=None, 
                                                layout_tipo='drl', vertex_size_factor=None, edge_width_factor=None):
    start_time = time.time()

    # Filtrar o DataFrame pelo setor
    df_setor = df[df['Setor'] == setor]

    # Criar o grafo e adicionar vértices com CNPJs
    G = ig.Graph(directed=False)
    cnpjs = df_setor['CNPJ'].tolist()
    G.add_vertices(cnpjs)

    # Mapeamento CNPJ -> índice
    cnpj_to_idx = {cnpj: idx for idx, cnpj in enumerate(cnpjs)}

    # Construir as arestas com barra de progresso e tempo restante
    edges, weights = [], []
    total_combinations = len(df_setor) * (len(df_setor) - 1) // 2  # Total de pares

    with tqdm(total=total_combinations, desc="Construindo arestas", unit="aresta") as pbar:
        for i, j in combinations(range(len(df_setor)), 2):
            peso = calcular_peso(df_setor.iloc[i], df_setor.iloc[j])
            if peso_minimo is None or peso >= peso_minimo:
                edges.append((cnpj_to_idx[df_setor.iloc[i]['CNPJ']], cnpj_to_idx[df_setor.iloc[j]['CNPJ']]))
                weights.append(peso)
            pbar.update(1)

    G.add_edges(edges)
    G.es['weight'] = weights

    # Simplificar o grafo e remover nós sem arestas
    G.simplify(multiple=True, loops=True, combine_edges=dict(weight="sum"))
    G.vs.select(_degree_eq=0).delete()

    # Aplicar filtro por grau mínimo
    if grau_minimo is not None:
        G.vs.select(_degree_lt=grau_minimo).delete()

    if G.vcount() == 0:
        print(f"Rede do setor {setor} está vazia.")
        return None

    # Atualizar a lista de CNPJs para os nós restantes
    cnpjs_restantes = [v["name"] for v in G.vs]

    # Calcular centralidades e comunidades
    degree_centrality = G.strength(weights='weight')  # Usar os pesos das arestas
    partition = G.community_multilevel(weights='weight')

    # Criar DataFrame com as métricas dos nós
    metricas_nodos = pd.DataFrame({
        'CNPJ': cnpjs_restantes,
        'Centralidade de Grau Ponderada': [deg / (G.vcount() - 1) for deg in degree_centrality],
        'Comunidade': partition.membership
    })

    # Salvar as métricas em Parquet
    metricas_nodos.to_parquet(f'metricas_{setor}.parquet', index=False)
    print(f"Métricas salvas como 'metricas_{setor}.parquet'.")

    # Calcular métricas globais antes da plotagem
    clustering_coefficient = G.transitivity_avglocal_undirected()
    
    if G.is_connected():
        average_path_length = G.average_path_length(weights='weight')
        diameter = G.diameter(weights='weight')  # Diâmetro com pesos
    else:
        largest_cluster = G.clusters().giant()
        average_path_length = largest_cluster.average_path_length(weights='weight')
        diameter = largest_cluster.diameter(weights='weight')  # Diâmetro para o maior cluster

    # Densidade da rede (não ponderada)
    density = G.density()

    # Exibir e salvar as métricas globais
    metrics = {
        'Setor': setor,
        'Número de Nós': G.vcount(),
        'Número de Arestas': G.ecount(),
        'Coeficiente de Agrupamento Médio': clustering_coefficient,
        'Comprimento Médio do Caminho (Ponderado)': average_path_length,
        'Diâmetro (Ponderado)': diameter,
        'Densidade': density,
        'Número de Comunidades': len(partition)
    }

    elapsed_time = time.time() - start_time
    print("\nMétricas da Rede:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

    # Salvar as métricas globais em um arquivo CSV e Parquet
    df_metrics = pd.DataFrame([metrics])
    df_metrics.to_parquet(f'metricas_globais_{setor}.parquet', index=False)
    print(f"Métricas globais salvas como 'metricas_globais_{setor}.parquet'.")

    # Plotar o grafo diretamente para o arquivo
    layout = G.layout(layout_tipo)
    cores_comunidades = plt.cm.get_cmap('viridis', len(partition))
    G.vs['color'] = [cores_comunidades(partition.membership[i] / len(partition)) for i in range(G.vcount())]

    # Calcular o valor máximo de centralidade
    max_centrality = max(degree_centrality)

# Normalizar os tamanhos dos nós e ajustar com o fator de escala
    vertex_size = [(deg / max_centrality) * vertex_size_factor * 100 for deg in degree_centrality]


    # Salvar o grafo diretamente como imagem
    ig.plot(
        G, 
        target=f'rede_setor_{setor}.png', 
        layout=layout, 
        vertex_size=vertex_size, 
        edge_width=[w * edge_width_factor for w in G.es['weight']],
        vertex_color=G.vs['color'],
        edge_color='gray', 
        bbox=(0, 0, 1000, 1000), 
        margin=50
    )
    print(f"Grafo salvo como 'rede_setor_{setor}.png'.")

    print(f"\nTempo de execução: {elapsed_time:.2f} segundos")

    return metrics

In [37]:
# Escolher um setor específico para análise
setor_especifico = "Agropecuaria"  # Substitua pelo setor desejado
layout_tipo = "kk"  # Escolher o layout desejado ('drl', 'fr', 'kk', 'lgl')

# Definir parâmetros adicionais
peso_minimo = 2  # Definir um peso mínimo para as arestas (opcional)
grau_minimo = 35  # Definir um grau mínimo para os nós (opcional)

# Definir os fatores de escala para os tamanhos dos nós e espessura das arestas
vertex_size_factor = 0.10 # Ajuste conforme o tamanho da rede
edge_width_factor = 0.03   # Ajuste conforme o peso das arestas

# Chamar a função e calcular as métricas para o setor escolhido
metrics = analisar_rede_por_setor_e_calcular_metricas(
    df_filtered, 
    setor_especifico, 
    peso_minimo=peso_minimo, 
    grau_minimo=grau_minimo, 
    layout_tipo=layout_tipo,
    vertex_size_factor=vertex_size_factor, 
    edge_width_factor=edge_width_factor
)

# Exibir as métricas da rede
if metrics is not None:
    print("\nMétricas do setor específico:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

Construindo arestas: 100%|██████████| 1771903/1771903 [08:34<00:00, 3441.50aresta/s]


Métricas salvas como 'metricas_Agropecuaria.parquet'.

Métricas da Rede:
Setor: Agropecuaria
Número de Nós: 1203
Número de Arestas: 88813
Coeficiente de Agrupamento Médio: 0.933615486642183
Comprimento Médio do Caminho (Ponderado): 5.557555086216793
Diâmetro (Ponderado): 12.0
Densidade: 0.12283904769413129
Número de Comunidades: 11
Métricas globais salvas como 'metricas_globais_Agropecuaria.parquet'.


C:\Users\fmarc\AppData\Local\Temp\ipykernel_22896\3134312234.py:122: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cores_comunidades = plt.cm.get_cmap('viridis', len(partition))


Grafo salvo como 'rede_setor_Agropecuaria.png'.

Tempo de execução: 520.57 segundos

Métricas do setor específico:
Setor: Agropecuaria
Número de Nós: 1203
Número de Arestas: 88813
Coeficiente de Agrupamento Médio: 0.933615486642183
Comprimento Médio do Caminho (Ponderado): 5.557555086216793
Diâmetro (Ponderado): 12.0
Densidade: 0.12283904769413129
Número de Comunidades: 11


In [46]:
# Escolher um setor específico para análise
setor_especifico = "Industria"  # Substitua pelo setor desejado
layout_tipo = "kk"  # Escolher o layout desejado ('drl', 'fr', 'kk', 'lgl')

# Definir parâmetros adicionais
peso_minimo = 2  # Definir um peso mínimo para as arestas (opcional)
grau_minimo = 35  # Definir um grau mínimo para os nós (opcional)

# Definir os fatores de escala para os tamanhos dos nós e espessura das arestas
vertex_size_factor = 0.08  # Ajuste conforme o tamanho da rede #tetar com 0.01 foi com 0.02
edge_width_factor = 0.03   # Ajuste conforme o peso das arestas

# Chamar a função e calcular as métricas para o setor escolhido
metrics = analisar_rede_por_setor_e_calcular_metricas(
    df_filtered, 
    setor_especifico, 
    peso_minimo=peso_minimo, 
    grau_minimo=grau_minimo, 
    layout_tipo=layout_tipo,
    vertex_size_factor=vertex_size_factor, 
    edge_width_factor=edge_width_factor
)

# Exibir as métricas da rede
if metrics is not None:
    print("\nMétricas do setor específico:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

Construindo arestas: 100%|██████████| 33574915/33574915 [2:19:17<00:00, 4017.23aresta/s]  


Métricas salvas como 'metricas_Industria.parquet'.

Métricas da Rede:
Setor: Industria
Número de Nós: 5873
Número de Arestas: 661095
Coeficiente de Agrupamento Médio: 0.7962263781360259
Comprimento Médio do Caminho (Ponderado): 6.272556580221408
Diâmetro (Ponderado): 12.0
Densidade: 0.03833962144223484
Número de Comunidades: 20
Métricas globais salvas como 'metricas_globais_Industria.parquet'.


C:\Users\fmarc\AppData\Local\Temp\ipykernel_22896\3134312234.py:122: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cores_comunidades = plt.cm.get_cmap('viridis', len(partition))


Grafo salvo como 'rede_setor_Industria.png'.

Tempo de execução: 8626.46 segundos

Métricas do setor específico:
Setor: Industria
Número de Nós: 5873
Número de Arestas: 661095
Coeficiente de Agrupamento Médio: 0.7962263781360259
Comprimento Médio do Caminho (Ponderado): 6.272556580221408
Diâmetro (Ponderado): 12.0
Densidade: 0.03833962144223484
Número de Comunidades: 20


In [ ]:
# Escolher um setor específico para análise
setor_especifico = "Construcao Civil"  # Substitua pelo setor desejado
layout_tipo = "kk"  # Escolher o layout desejado ('drl', 'fr', 'kk', 'lgl')

# Definir parâmetros adicionais
peso_minimo = 2  # Definir um peso mínimo para as arestas (opcional)
grau_minimo = 35  # Definir um grau mínimo para os nós (opcional)

# Definir os fatores de escala para os tamanhos dos nós e espessura das arestas
vertex_size_factor = 0.005  # Ajuste conforme o tamanho da rede #tetar com 0.01 foi com 0.02
edge_width_factor = 0.03  # Ajuste conforme o peso das arestas

# Chamar a função e calcular as métricas para o setor escolhido
metrics = analisar_rede_por_setor_e_calcular_metricas(
    df_filtered, 
    setor_especifico, 
    peso_minimo=peso_minimo, 
    grau_minimo=grau_minimo, 
    layout_tipo=layout_tipo,
    vertex_size_factor=vertex_size_factor, 
    edge_width_factor=edge_width_factor
)

# Exibir as métricas da rede
if metrics is not None:
    print("\nMétricas do setor específico:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

In [ ]:
import pandas as pd
import igraph as ig
import time
from itertools import combinations
from tqdm import tqdm

# Função para calcular o peso das arestas
def calcular_peso(row1, row2):
    peso = 0
    if row1['CNAE_Principal'] == row2['CNAE_Principal']:
        peso += 1
    if row1['MUNICIPIO'] == row2['MUNICIPIO']:
        peso += 1
    if row1['Porte'] == row2['Porte']:
        peso += 1
    if row1['COD_RGI'] == row2['COD_RGI']:
        peso += 0.5
    if row1['CNAE_Divisão'] == row2['CNAE_Divisão']:
        peso += 0.5
    return peso

# Função principal para análise da rede
def analisar_rede_por_setor_e_calcular_metricas(df, setor, peso_minimo=None, grau_minimo=None, 
                                                layout_tipo='drl', vertex_size_factor=0.01, edge_width_factor=0.01):
    start_time = time.time()

    # Filtrar o DataFrame pelo setor
    df_setor = df[df['Setor'] == setor]

    # Criar uma amostra aleatória de 10000 empresas
    df_setor = df_setor.sample(n=13000, random_state=42)

    # Criar o grafo e adicionar vértices com CNPJs
    G = ig.Graph(directed=False)
    cnpjs = df_setor['CNPJ'].tolist()
    G.add_vertices(cnpjs)

    # Mapeamento CNPJ -> índice
    cnpj_to_idx = {cnpj: idx for idx, cnpj in enumerate(cnpjs)}

    # Construir as arestas com barra de progresso e tempo restante
    edges, weights = [], []
    total_combinations = len(df_setor) * (len(df_setor) - 1) // 2  # Total de pares

    with tqdm(total=total_combinations, desc="Construindo arestas", unit="aresta") as pbar:
        for i, j in combinations(range(len(df_setor)), 2):
            peso = calcular_peso(df_setor.iloc[i], df_setor.iloc[j])
            if peso_minimo is None or peso >= peso_minimo:
                edges.append((cnpj_to_idx[df_setor.iloc[i]['CNPJ']], cnpj_to_idx[df_setor.iloc[j]['CNPJ']]))
                weights.append(peso)
            pbar.update(1)

    G.add_edges(edges)
    G.es['weight'] = weights

    # Simplificar o grafo e remover nós sem arestas
    G.simplify(multiple=True, loops=True, combine_edges=dict(weight="sum"))
    G.vs.select(_degree_eq=0).delete()

    # Aplicar filtro por grau mínimo
    if grau_minimo is not None:
        G.vs.select(_degree_lt=grau_minimo).delete()

    if G.vcount() == 0:
        print(f"Rede do setor {setor} está vazia.")
        return None

    # Atualizar a lista de CNPJs para os nós restantes
    cnpjs_restantes = [v["name"] for v in G.vs]

    # Calcular centralidades e comunidades
    degree_centrality = G.strength(weights='weight')  # Usar os pesos das arestas
    partition = G.community_multilevel(weights='weight')

    # Criar DataFrame com as métricas dos nós
    metricas_nodos = pd.DataFrame({
        'CNPJ': cnpjs_restantes,
        'Centralidade de Grau Ponderada': [deg / (G.vcount() - 1) for deg in degree_centrality],
        'Comunidade': partition.membership
    })

    # Salvar as métricas em Parquet
    metricas_nodos.to_parquet(f'metricas_{setor}.parquet', index=False)
    print(f"Métricas salvas como 'metricas_{setor}.parquet'.")

    # Calcular métricas globais antes da plotagem
    clustering_coefficient = G.transitivity_avglocal_undirected()
    
    if G.is_connected():
        average_path_length = G.average_path_length(weights='weight')
        diameter = G.diameter(weights='weight')  # Diâmetro com pesos
    else:
        largest_cluster = G.clusters().giant()
        average_path_length = largest_cluster.average_path_length(weights='weight')
        diameter = largest_cluster.diameter(weights='weight')  # Diâmetro para o maior cluster

    # Densidade da rede (não ponderada)
    density = G.density()

    # Exibir e salvar as métricas globais
    metrics = {
        'Setor': setor,
        'Número de Nós': G.vcount(),
        'Número de Arestas': G.ecount(),
        'Coeficiente de Agrupamento Médio': clustering_coefficient,
        'Comprimento Médio do Caminho (Ponderado)': average_path_length,
        'Diâmetro (Ponderado)': diameter,
        'Densidade': density,
        'Número de Comunidades': len(partition)
    }

    elapsed_time = time.time() - start_time
    print("\nMétricas da Rede:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

    # Salvar as métricas globais em um arquivo CSV e Parquet
    df_metrics = pd.DataFrame([metrics])
    df_metrics.to_parquet(f'metricas_globais_{setor}.parquet', index=False)
    print(f"Métricas globais salvas como 'metricas_globais_{setor}.parquet'.")

    # Plotar o grafo diretamente para o arquivo
    layout = G.layout(layout_tipo)
    cores_comunidades = plt.cm.get_cmap('viridis', len(partition))
    G.vs['color'] = [cores_comunidades(partition.membership[i] / len(partition)) for i in range(G.vcount())]

    # Salvar o grafo diretamente como imagem
    ig.plot(
        G, 
        target=f'rede_setor_{setor}.png', 
        layout=layout, 
        vertex_size=[deg * vertex_size_factor for deg in degree_centrality], 
        edge_width=[w * edge_width_factor for w in G.es['weight']],
        vertex_color=G.vs['color'],
        edge_color='gray', 
        bbox=(0, 0, 1000, 1000), 
        margin=50
    )
    print(f"Grafo salvo como 'rede_setor_{setor}.png'.")

    print(f"\nTempo de execução: {elapsed_time:.2f} segundos")

    return metrics

In [ ]:
# Escolher um setor específico para análise
setor_especifico = "Comercio"  # Substitua pelo setor desejado
layout_tipo = "kk"  # Escolher o layout desejado ('drl', 'fr', 'kk', 'lgl')

# Definir parâmetros adicionais
peso_minimo = 2  # Definir um peso mínimo para as arestas (opcional)
grau_minimo = 35  # Definir um grau mínimo para os nós (opcional)

# Definir os fatores de escala para os tamanhos dos nós e espessura das arestas
vertex_size_factor = 0.01  # Ajuste conforme o tamanho da rede #tetar com 0.01 foi com 0.02
edge_width_factor = 0.02   # Ajuste conforme o peso das arestas

# Chamar a função e calcular as métricas para o setor escolhido
metrics = analisar_rede_por_setor_e_calcular_metricas(
    df_filtered, 
    setor_especifico, 
    peso_minimo=peso_minimo, 
    grau_minimo=grau_minimo, 
    layout_tipo=layout_tipo,
    vertex_size_factor=vertex_size_factor, 
    edge_width_factor=edge_width_factor
)

# Exibir as métricas da rede 
if metrics is not None:
    print("\nMétricas do setor específico:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

In [ ]:
import pandas as pd
import igraph as ig
import time
from itertools import combinations
from tqdm import tqdm

# Função para calcular o peso das arestas
def calcular_peso(row1, row2):
    peso = 0
    if row1['CNAE_Principal'] == row2['CNAE_Principal']:
        peso += 1
    if row1['MUNICIPIO'] == row2['MUNICIPIO']:
        peso += 1
    if row1['Porte'] == row2['Porte']:
        peso += 1
    if row1['COD_RGI'] == row2['COD_RGI']:
        peso += 0.5
    if row1['CNAE_Divisão'] == row2['CNAE_Divisão']:
        peso += 0.5
    return peso

# Função principal para análise da rede
def analisar_rede_por_setor_e_calcular_metricas(df, setor, peso_minimo=None, grau_minimo=None, 
                                                layout_tipo='drl', vertex_size_factor=0.01, edge_width_factor=0.01):
    start_time = time.time()

    # Filtrar o DataFrame pelo setor
    df_setor = df[df['Setor'] == setor]

    # Criar uma amostra aleatória de 10000 empresas
    df_setor = df_setor.sample(n=17000, random_state=42)

    # Criar o grafo e adicionar vértices com CNPJs
    G = ig.Graph(directed=False)
    cnpjs = df_setor['CNPJ'].tolist()
    G.add_vertices(cnpjs)

    # Mapeamento CNPJ -> índice
    cnpj_to_idx = {cnpj: idx for idx, cnpj in enumerate(cnpjs)}

    # Construir as arestas com barra de progresso e tempo restante
    edges, weights = [], []
    total_combinations = len(df_setor) * (len(df_setor) - 1) // 2  # Total de pares

    with tqdm(total=total_combinations, desc="Construindo arestas", unit="aresta") as pbar:
        for i, j in combinations(range(len(df_setor)), 2):
            peso = calcular_peso(df_setor.iloc[i], df_setor.iloc[j])
            if peso_minimo is None or peso >= peso_minimo:
                edges.append((cnpj_to_idx[df_setor.iloc[i]['CNPJ']], cnpj_to_idx[df_setor.iloc[j]['CNPJ']]))
                weights.append(peso)
            pbar.update(1)

    G.add_edges(edges)
    G.es['weight'] = weights

    # Simplificar o grafo e remover nós sem arestas
    G.simplify(multiple=True, loops=True, combine_edges=dict(weight="sum"))
    G.vs.select(_degree_eq=0).delete()

    # Aplicar filtro por grau mínimo
    if grau_minimo is not None:
        G.vs.select(_degree_lt=grau_minimo).delete()

    if G.vcount() == 0:
        print(f"Rede do setor {setor} está vazia.")
        return None

    # Atualizar a lista de CNPJs para os nós restantes
    cnpjs_restantes = [v["name"] for v in G.vs]

    # Calcular centralidades e comunidades
    degree_centrality = G.strength(weights='weight')  # Usar os pesos das arestas
    partition = G.community_multilevel(weights='weight')

    # Criar DataFrame com as métricas dos nós
    metricas_nodos = pd.DataFrame({
        'CNPJ': cnpjs_restantes,
        'Centralidade de Grau Ponderada': [deg / (G.vcount() - 1) for deg in degree_centrality],
        'Comunidade': partition.membership
    })

    # Salvar as métricas em Parquet
    metricas_nodos.to_parquet(f'metricas_{setor}.parquet', index=False)
    print(f"Métricas salvas como 'metricas_{setor}.parquet'.")

    # Calcular métricas globais antes da plotagem
    clustering_coefficient = G.transitivity_avglocal_undirected()
    
    if G.is_connected():
        average_path_length = G.average_path_length(weights='weight')
        diameter = G.diameter(weights='weight')  # Diâmetro com pesos
    else:
        largest_cluster = G.clusters().giant()
        average_path_length = largest_cluster.average_path_length(weights='weight')
        diameter = largest_cluster.diameter(weights='weight')  # Diâmetro para o maior cluster

    # Densidade da rede (não ponderada)
    density = G.density()

    # Exibir e salvar as métricas globais
    metrics = {
        'Setor': setor,
        'Número de Nós': G.vcount(),
        'Número de Arestas': G.ecount(),
        'Coeficiente de Agrupamento Médio': clustering_coefficient,
        'Comprimento Médio do Caminho (Ponderado)': average_path_length,
        'Diâmetro (Ponderado)': diameter,
        'Densidade': density,
        'Número de Comunidades': len(partition)
    }

    elapsed_time = time.time() - start_time
    print("\nMétricas da Rede:")
    for key, value in metrics.items():
        print(f"{key}: {value}")

    # Salvar as métricas globais em um arquivo CSV e Parquet
    df_metrics = pd.DataFrame([metrics])
    df_metrics.to_parquet(f'metricas_globais_{setor}.parquet', index=False)
    print(f"Métricas globais salvas como 'metricas_globais_{setor}.parquet'.")

    # Plotar o grafo diretamente para o arquivo
    layout = G.layout(layout_tipo)
    cores_comunidades = plt.cm.get_cmap('viridis', len(partition))
    G.vs['color'] = [cores_comunidades(partition.membership[i] / len(partition)) for i in range(G.vcount())]

    # Salvar o grafo diretamente como imagem
    ig.plot(
        G, 
        target=f'rede_setor_{setor}.png', 
        layout=layout, 
        vertex_size=[deg * vertex_size_factor for deg in degree_centrality], 
        edge_width=[w * edge_width_factor for w in G.es['weight']],
        vertex_color=G.vs['color'],
        edge_color='gray', 
        bbox=(0, 0, 1000, 1000), 
        margin=50
    )
    print(f"Grafo salvo como 'rede_setor_{setor}.png'.")

    print(f"\nTempo de execução: {elapsed_time:.2f} segundos")

    return metrics

In [ ]:
# Escolher um setor específico para análise
setor_especifico = "Servicos"  # Substitua pelo setor desejado
layout_tipo = "kk"  # Escolher o layout desejado ('drl', 'fr', 'kk', 'lgl')

# Definir parâmetros adicionais
peso_minimo = 2  # Definir um peso mínimo para as arestas (opcional)
grau_minimo = 35  # Definir um grau mínimo para os nós (opcional)

# Definir os fatores de escala para os tamanhos dos nós e espessura das arestas
vertex_size_factor = 0.01  # Ajuste conforme o tamanho da rede #tetar com 0.01 foi com 0.02
edge_width_factor = 0.02   # Ajuste conforme o peso das arestas

# Chamar a função e calcular as métricas para o setor escolhido
metrics = analisar_rede_por_setor_e_calcular_metricas(
    df_filtered, 
    setor_especifico, 
    peso_minimo=peso_minimo, 
    grau_minimo=grau_minimo, 
    layout_tipo=layout_tipo,
    vertex_size_factor=vertex_size_factor, 
    edge_width_factor=edge_width_factor
)

# Exibir as métricas da rede
if metrics is not None:
    print("\nMétricas do setor específico:")
    for key, value in metrics.items():
        print(f"{key}: {value}")